## Pytorch Getting Start
Basic CNN with Pytorch

In [56]:
import torch
import torchvision
import torchvision.transforms as transforms 
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


Basic With Pytorch Tensor

In [57]:
tensor_1d = torch.tensor([1,2,3])
print("Tensor 1D (Vector)")
print(tensor_1d)
print('-----')

tensor_2d = torch.tensor([[1,2],[3,4]])
print('Tensor 2D (Vector)')
print(tensor_2d)

zeros_tensor = torch.zeros(2,3)
print('Zero Tensors with dimension 2,3')
print(zeros_tensor)

Tensor 1D (Vector)
tensor([1, 2, 3])
-----
Tensor 2D (Vector)
tensor([[1, 2],
        [3, 4]])
Zero Tensors with dimension 2,3
tensor([[0., 0., 0.],
        [0., 0., 0.]])


Loading the basic CIFAR-10 dataset

In [58]:
# Create the transformer that we will use to transform the input data to Tensor and normalize them

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5)) #Normalize to [-1,1]
])

trainset = torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform= transform)
train_loader = torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True,num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(testset,batch_size=4,shuffle=False,num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Model Defining
Defining the CNN Model which include the Conv Layer, Pooling Layer and Fully Connected Layer

In [59]:
# Defining the CNN Model extending the pytorch NN Model
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork,self).__init__()
        # Does not specify the input size for height and width; 
        # the input must be a tensor of shape (batch_size, 3, height, width), where the height and width can be any valid dimension.
        self.conv1 = nn.Conv2d(in_channels = 3,out_channels = 6,kernel_size =5)
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv2 = nn.Conv2d(in_channels = 6,out_channels=16,kernel_size=5)
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self,x):
        ## First Step Value
        x = self.conv1(x)
        # Take the activation layer
        x = torch.relu(x)
        # Take the pooling layer
        x = self.pool(x)

        ## Second Step 
        # ConvNet
        x = self.conv2(x)
        # Activation
        x = torch.relu(x)
        # Pooling Layer
        x = self.pool(x)

        x = x.view(-1,16*5*5)

        # Taking into the fully-connected layer
        x = self.fc1(x)
        # Activation
        x = torch.relu(x)

        # Take the second FC Layer short hand
        x = torch.relu(self.fc2(x))

        # Take the final fc layer which activated to the class
        x = self.fc3(x)
        return x

net = MyNetwork() 

        

Visualize the model

In [60]:
writer = SummaryWriter('runs/basic_experiment')

Define the loss function and optimizer

In [61]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,momentum=0.9)

Model Training Process

In [62]:
for epoch in range(2):
    running_loss  = 0.0

    for i,data in enumerate(train_loader,0):
        # Loading the input i 
        inputs, labels = data
        
        # Set the gradient to none
        optimizer.zero_grad()

        # Take the input into model and Recieve the output 
        outputs = net(inputs)

        # Analysis of output using cross entrophy
        loss = criterion(outputs,labels)
        
        # Backward Propagation
        loss.backward()

        # Perform the optimizer 1 step for reduce the loss
        optimizer.step()

        # sum of each lose
        running_loss += loss.item()
        
        # Print Every 2000 Mini batchs
        if i % 2000 == 1999:
            # Find the average loss
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 2000))

            # Clear the loss
            running_loss = 0.0

print('Finish Training')



[1,  2000] loss: 2.184
[1,  4000] loss: 1.882
[1,  6000] loss: 1.691
[1,  8000] loss: 1.582
[1, 10000] loss: 1.541
[1, 12000] loss: 1.479
[2,  2000] loss: 1.399
[2,  4000] loss: 1.379
[2,  6000] loss: 1.361
[2,  8000] loss: 1.330
[2, 10000] loss: 1.289
[2, 12000] loss: 1.287
Finish Training


In [63]:
# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# create grid of images
img_grid = torchvision.utils.make_grid(images)

# write to tensorboard
writer.add_image('basic_experiment', img_grid)